In [72]:
import os
import numpy as np
from matplotlib import pyplot
# Modulo de optimización
from scipy import optimize
%matplotlib inline

## Cargar datos
### Dataset
Atributos del dataset:
1. Marca
2. Modelo (Ej: 2021, 2016)
3. Número de puertas
4. Número de asientos
5. Condición del vehículo
6. Tipo
8. Transmisión
9. Tipo de Combustible
7. Kilometraje (recorrido en Km)
10. Color
11. Procedencia
12. Cilindrada (en cc)
13. Estado de la pintura
14. ¿La pintura está en buen estado?
15. ¿El vehículo tuvo algún accidente?
16. ¿El vehículo cuenta con todos los accesorios?
17. Costo en $us
18. Tipo de costo (caro=1; algo caro=2; accesible=3; muy accesible=4; barato=5)


In [73]:
data_train = np.loadtxt('DVO.csv', delimiter=',', dtype=float)
data_test = np.loadtxt('prueba.csv', delimiter=',', dtype=float)
#print(data_train)
# print(data_test)

### X: Marca, año de fabricación, Cilindrada, Kilometraje, Tipo de Movilidad y estado interior y exterior

In [74]:
# X = data_train[:,0:17]
X = data_train[:,[0, 1, 4, 5, 8, 11, 15]]
# X_test = data_test[:,0:17]
X_test = data_test[:,[0, 1, 4, 5, 8, 11, 15]]
print(X)
#print(X_test)

[[1.20000e+01 2.01700e+03 3.00000e+00 ... 4.23970e+04 1.20000e+03
  2.00000e+00]
 [1.90000e+01 2.01900e+03 2.00000e+00 ... 0.00000e+00 3.70000e+03
  2.00000e+00]
 [2.00000e+00 2.01800e+03 2.00000e+00 ... 3.80000e+04 2.70000e+03
  2.00000e+00]
 ...
 [1.90000e+01 2.00300e+03 3.00000e+00 ... 6.40000e+04 1.60000e+03
  1.00000e+00]
 [3.50000e+01 1.98000e+03 4.00000e+00 ... 2.00000e+05 1.50000e+03
  1.00000e+00]
 [1.70000e+01 1.99200e+03 3.00000e+00 ... 2.65918e+05 2.50000e+03
  1.00000e+00]]


In [75]:
y = data_train[:,17].astype(int)
# y = np.asarray(y, dtype=np.int)
y_test = data_test[:,17].astype(int)
# y_test = np.asarray(y_test, dtype=np.int)

X.shape, y.shape


((800, 7), (800,))

In [76]:
print('X \n',X,'\n Y\n',y)

X 
 [[1.20000e+01 2.01700e+03 3.00000e+00 ... 4.23970e+04 1.20000e+03
  2.00000e+00]
 [1.90000e+01 2.01900e+03 2.00000e+00 ... 0.00000e+00 3.70000e+03
  2.00000e+00]
 [2.00000e+00 2.01800e+03 2.00000e+00 ... 3.80000e+04 2.70000e+03
  2.00000e+00]
 ...
 [1.90000e+01 2.00300e+03 3.00000e+00 ... 6.40000e+04 1.60000e+03
  1.00000e+00]
 [3.50000e+01 1.98000e+03 4.00000e+00 ... 2.00000e+05 1.50000e+03
  1.00000e+00]
 [1.70000e+01 1.99200e+03 3.00000e+00 ... 2.65918e+05 2.50000e+03
  1.00000e+00]] 
 Y
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

### Funcion activacion relu
La función relu transforma los valores introducidos anulando los valores negativos y dejando los positivos tal y como entran

In [77]:
#funciondes de activacion
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

### En cuanto a las funciones de activación que utilizaremos a la salida del MLP

Lineal: usada para regresión (junto a la función de pérdida MSE).

Sigmoid: usada para clasificación binaria (junto a la función de pérdida BCE).

Softmax: usada para clasificación multiclase (junto a la función de pérdida crossentropy, CE).

In [78]:
def linear(x):
    return x

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

### funciones de perdida

In [79]:
# Mean Square Error -> usada para regresión (con activación lineal)
def mse(y, y_hat):
    return np.mean((y_hat - y.reshape(y_hat.shape))**2)

# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

# Cross Entropy (aplica softmax + cross entropy de manera estable) -> usada para clasificación multiclase
def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

Derivadas

In [80]:
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1    
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

In [81]:
# clase base MLP

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2
    return self.activation(y_hat)

  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size]
            # salida del perceptrón
            y_pred = self(x)
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)
            # Backprop
            dldy = self.grad_loss(y, y_pred)
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [82]:
# MLP para regresión
class MLPRegression(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, mse, grad_mse, linear)

# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

In [83]:
# normalización datos
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_norm = (X - X_mean) / X_std

In [84]:
model = MLPClassification(D_in=7, H=100, D_out=6)
epochs = 1000
lr = 0.2
model.fit(X_norm, y, epochs, lr, batch_size=100, log_each=100)

Epoch: 100/1000, Loss: 1.09460
Epoch: 200/1000, Loss: 1.04160
Epoch: 300/1000, Loss: 1.01398
Epoch: 400/1000, Loss: 0.99605
Epoch: 500/1000, Loss: 0.98332
Epoch: 600/1000, Loss: 0.97337
Epoch: 700/1000, Loss: 0.96520
Epoch: 800/1000, Loss: 0.95786
Epoch: 900/1000, Loss: 0.95114
Epoch: 1000/1000, Loss: 0.94490


In [85]:
y_predict_ = model.predict(model.ws[999], X_norm)
y_pred_train=[]
for i in y_predict_:
    y_pred_train.append(np.argmax(i))
    print(np.argmax(i))

4
1
1
4
2
2
1
1
1
1
2
2
2
1
2
1
1
2
2
1
2
1
2
2
2
2
2
1
1
2
3
1
1
1
1
3
2
2
2
1
2
1
1
2
1
1
2
1
2
1
1
1
2
2
3
2
2
1
2
1
1
2
1
2
2
2
2
2
2
1
2
1
1
1
1
1
1
1
1
2
3
2
2
2
2
2
1
1
1
2
2
1
2
2
1
1
1
2
1
2
1
1
2
2
2
2
3
1
2
2
2
2
1
2
2
2
2
1
2
1
2
2
2
2
2
2
2
2
2
2
1
2
1
1
2
1
2
2
2
1
2
2
1
2
1
1
4
1
1
1
2
1
1
1
2
3
1
2
1
1
2
2
2
2
2
1
2
1
1
2
2
1
1
3
2
2
1
3
2
2
1
3
3
2
3
2
3
2
2
2
2
1
2
2
2
2
2
1
1
1
1
2
2
1
1
1
2
2
2
2
2
1
2
1
2
2
1
1
1
2
2
3
3
2
3
2
3
1
3
2
2
2
2
2
2
2
2
2
3
1
3
2
1
2
2
2
3
2
2
2
3
3
3
2
2
3
2
3
3
3
2
2
2
2
2
2
2
2
2
2
3
2
2
3
2
2
3
2
2
3
2
3
2
2
2
3
2
2
2
2
2
2
3
2
2
4
3
3
3
2
2
3
3
2
3
3
3
2
2
2
2
3
2
2
4
3
2
2
2
2
2
3
2
3
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
3
2
2
2
2
1
1
2
2
2
3
2
2
3
2
3
2
2
2
2
2
2
2
3
2
2
2
3
2
4
2
2
2
4
4
2
2
4
2
1
2
2
1
2
2
2
2
2
2
2
2
2
3
2
3
2
2
2
2
3
4
3
2
2
2
3
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
4
2
2
3
2
3
1
3
2
2
1
2
3
2
2
3
2
2
2
2
3
2
2
3
2
2
3
2
3
2
2
2
2
2
4
2
4
3
2
3
3
2
2
3
2
2
1
2
2
2
2
2
2
3
3
1
2
2
3
2
3
3
3


precicion con datos te entrenamiento

In [86]:
def accuracy(y_pred, y):
    return np.sum(y_pred == y) / len(y)

print('Precision de entrenamiento: {:.2f}%'.format(accuracy(y_pred_train, y)*100))

Precision de entrenamiento: 62.38%


### Datos de prueba

In [87]:
y_test


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4])

In [88]:
X_norm_test = (X_test - X_mean) / X_std
y_predict = model.predict(model.ws[999], X_norm_test)
y_predict_test=[]
for i in y_predict:
    y_predict_test.append(np.argmax(i))
    print(np.argmax(i))

1
2
2
2
2
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
3
2
2
2
3
3
3
3
4
2
3
3
3
4
3
3
4
3
4
2
4
4
2
2
3
4
3
3
3
3
3
3
2
4
2
1
2
2
1
2
2
2
2
2
4
4
4
4
4
4
4
1
4
4
4
4
4
4
4
4
4
4
4
2
4
4


In [89]:

print('Precision de prueba: {:.2f}%'.format(accuracy(y_predict_test,y_test)*100))

Precision de prueba: 65.56%
